In [2]:
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [3]:
# face Detection
def detect_and_save_faces(image_path, output_folder):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Error loading image {image_path}")
        return
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Create the output directory if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    for i, (x, y, w, h) in enumerate(faces):
        face_img = img[y:y+h, x:x+w]
        output_path = Path(output_folder) / f"face_{i}_{image_path.name}"
        cv2.imwrite(str(output_path), face_img)
       


In [5]:
dataset_folder = '/Users/omerkhalid/Downloads/celeba_hq_256'
output_dir = '/Users/omerkhalid/Desktop/GenAI-DeepFake/Detected_faces'

for image_file in Path(dataset_folder).glob('*.jpg'):
    detect_and_save_faces(image_file, output_dir)


KeyboardInterrupt: 

In [7]:
!pip install dlib imutils


  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25837 sha256=f2ac17188d84b88ce7eecba5884e8883f4ddc5058a726617cdb4a1271fb7aaeb
  Stored in directory: /Users/omerkhalid/Library/Caches/pip/wheels/5b/76/96/ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils


In [9]:
import os
import cv2
import dlib
from imutils import face_utils
from pathlib import Path  # Import the Path class

# Initialize dlib's face detector (HOG-based) and create the facial landmark predictor
p = "/Users/omerkhalid/Desktop/GenAI-DeepFake/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Path to the directory containing images
folder_path = "/Users/omerkhalid/Desktop/GenAI-DeepFake/Detected_faces"
output_folder_path = "/Users/omerkhalid/Desktop/GenAI-DeepFake/aligned_face"  # Directory to save processed images

# Ensure the output directory exists using Path
Path(output_folder_path).mkdir(parents=True, exist_ok=True)

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
        # Construct full file path using Path for better path handling
        file_path = Path(folder_path) / filename
        
        # Load the input image and convert it to grayscale
        image = cv2.imread(str(file_path))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the grayscale image
        rects = detector(gray, 0)
        
        # Loop over the face detections
        for (i, rect) in enumerate(rects):
            # Determine the facial landmarks for the face region, then convert to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            
            # Loop over the (x, y)-coordinates for the facial landmarks and draw them on the image
            for (x, y) in shape:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        
        # Construct output file path using Path
        output_file_path = Path(output_folder_path) / filename
        
        # Save the output image with the face detections + facial landmarks
        cv2.imwrite(str(output_file_path), image)

print("Processing complete. All images saved to", output_folder_path)


Processing complete. All images saved to /Users/omerkhalid/Desktop/GenAI-DeepFake/aligned_face


In [16]:
import os
import cv2
import numpy as np
import dlib
from imutils import face_utils
from pathlib import Path

# Initialize dlib's face detector (HOG-based) and create the facial landmark predictor
p = "/Users/omerkhalid/Desktop/GenAI-DeepFake/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

# Path to the directory containing images
folder_path = "/Users/omerkhalid/Desktop/GenAI-DeepFake/Detected_faces"
output_folder_path = "/Users/omerkhalid/Desktop/GenAI-DeepFake/Segment1"  # Directory to save processed images

# Ensure the output directory exists using Path
Path(output_folder_path).mkdir(parents=True, exist_ok=True)

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
        # Construct full file path using Path for better path handling
        file_path = Path(folder_path) / filename
        
        # Load the input image and convert it to grayscale
        image = cv2.imread(str(file_path))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the grayscale image
        rects = detector(gray, 0)
        
        # Loop over the face detections
        for (i, rect) in enumerate(rects):
            # Determine the facial landmarks for the face region, then convert to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            
            # Create a mask for the face including eyes and jawline
            mask = np.zeros_like(gray)  # Create a mask the same size as the face
            # Define points for the jawline and eyes
            jawline_points = shape[0:17]  # Jaw line
            left_eye_points = shape[36:42]  # Left eye
            right_eye_points = shape[42:48]  # Right eye
            mask_points = np.concatenate([jawline_points, left_eye_points, right_eye_points])
            cv2.fillPoly(mask, [mask_points], 255)
            
            # Extract the face by bitwise and operation with the mask
            face = cv2.bitwise_and(image, image, mask=mask)
            
            # Save just the face part
            (x, y, w, h) = cv2.boundingRect(cv2.cvtColor(face, cv2.COLOR_BGR2GRAY))
            face_only = face[y:y+h, x:x+w]

            # Construct output file path using Path
            output_file_path = Path(output_folder_path) / f"face_{i}_{filename}"
            
            # Save the output image with the face segmentations
            cv2.imwrite(str(output_file_path), face_only)

print("Processing complete. All images saved to", output_folder_path)


KeyboardInterrupt: 